In [1]:
import os
import pandas as pd
from google.cloud import bigquery
import db_dtypes

#scenic-outcome-453219-h5-4c318c9a7175
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="scenic-outcome-453219-h5-4c318c9a7175.json"
client = bigquery.Client() 

pd.set_option('display.max_columns', None)  # wyłącz limit na kolumny
pd.set_option('display.max_rows', None)  # wyłącz limit na wiersze
pd.set_option('display.width', None)  # wyłącz limit na szerokość
pd.set_option('display.max_colwidth', None)  # wyłącz limit na szerokość kolumn
# client = bigquery.Client()

In [33]:
query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 5')
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()

In [34]:
df

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,cumulative_tested,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,population,population_male,population_female,population_rural,population_urban,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,gdp_usd,gdp_per_capita_usd,openstreetmap_id,latitude,longitude,area_sq_km,smoking_prevalence,diabetes_prevalence,infant_mortality_rate,nurses_per_1000,physicians_per_1000,health_expenditure_usd,out_of_pocket_health_expenditure_usd,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,facial_coverings,vaccination_policy,stringency_index,average_temperature_celsius,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,snowfall_mm,dew_point,relative_humidity,new_tested,population_largest_city,population_clustered,human_capital_index,area_rural_sq_km,area_urban_sq_km,life_expectancy,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,hospital_beds_per_1000,subregion1_code,subregion1_name,new_recovered,cumulative_recovered,elevation_m,new_hospitalized_patients,cumulative_hospitalized_patients,new_intensive_care_patients,cumulative_intensive_care_patients,new_confirmed_age_0,new_confirmed_age_1,new_confirmed_age_2,new_confirmed_age_3,new_confirmed_age_4,new_confirmed_age_5,new_confirmed_age_6,new_confirmed_age_7,new_confirmed_age_8,new_confirmed_age_9,cumulative_confirmed_age_0,cumulative_confirmed_age_1,cumulative_confirmed_age_2,cumulative_confirmed_age_3,cumulative_confirmed_age_4,cumulative_confirmed_age_5,cumulative_confirmed_age_6,cumulative_confirmed_age_7,cumulative_confirmed_age_8,cumulative_confirmed_age_9,new_deceased_age_0,new_deceased_age_1,new_deceased_age_2,new_deceased_age_3,new_deceased_age_4,new_deceased_age_5,new_deceased_age_6,new_deceased_age_7,new_deceased_age_8,new_deceased_age_9,cumulative_deceased_age_0,cumulative_deceased_age_1,cumulative_deceased_age_2,cumulative_deceased_age_3,cumulative_deceased_age_4,cumulative_deceased_age_5,cumulative_deceased_age_6,cumulative_deceased_age_7,cumulative_deceased_age_8,cumulative_deceased_age_9,new_tested_age_0,new_tested_age_1,new_tested_age_2,new_tested_age_3,new_tested_age_4,new_tested_age_5,new_tested_age_6,new_tested_age_7,new_tested_age_8,new_tested_age_9,cumulative_tested_age_0,cumulative_tested_age_1,cumulative_tested_age_2,cumulative_tested_age_3,cumulative_tested_age_4,cumulative_tested_age_5,cumulative_tested_age_6,cumulative_tested_age_7,cumulative_tested_age_8,cumulative_tested_age_9,new_hospitalized_patients_age_0,new_hospitalized_patients_age_1,new_hospitalized_patients_age_2,new_hospitalized_patients_age_3,new_hospitalized_patients_age_4,new_hospitalized_patients_age_5,new_hospitalized_patients_age_6,new_hospitalized_patients_age_7,new_hospitalized_patients_age_8,new_hospitalized_patients_age_9,cumulative_hospitalized_patients_age_0,cumulative_hospitalized_patients_age_1,cumulative_hospitalized_patients_age_2,cumulative_hospitalized_patients_age_3,cumulative_hospitalized_patients_age_4,cumulative_hospitalized_patie

In [35]:
#Lista wszystkich kolumn
query = """
    SELECT column_name
    FROM `bigquery-public-data.covid19_open_data.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = 'covid19_open_data'
    limit 5
"""
query_job = client.query(query)
columns_df = query_job.result().to_dataframe()

columns_df

,column_name
0,location_key
1,date
2,place_id
3,wikidata_id
4,datacommons_id


In [36]:
# 3.1. Sprawdź, ile jest zapisanych wierszy z danymi.
query = "SELECT COUNT(*) AS row_count FROM `bigquery-public-data.covid19_open_data.covid19_open_data`"
query_job = client.query(query)
rows_df = query_job.result().to_dataframe()
rows_df

,row_count
0,22756333


In [37]:
# 3.2. Sprawdź, ile krajów jest uwzględnionych w danych. 
query = "SELECT COUNT(DISTINCT country_name) AS country_count FROM `bigquery-public-data.covid19_open_data.covid19_open_data`"
query_job = client.query(query)
countries_df = query_job.result().to_dataframe()
countries_df

,country_count
0,246


In [38]:
# # 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.
#
# # Do pomocy, potem nadpisywane
# query = """
#     SELECT country_name, date, COUNT(*) AS records_per_day
#     FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
#     WHERE country_name = 'Poland' AND new_confirmed IS NOT NULL
#     GROUP BY country_name, date
#     ORDER BY country_name, date
#     LIMIT 10
# """
#
# query = """
#     SELECT country_name, date, new_confirmed, new_deceased, cumulative_confirmed, cumulative_deceased, cumulative_tested
#     FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
#     WHERE country_name = 'France' AND new_confirmed IS NOT NULL AND cumulative_confirmed != 0
#     ORDER BY country_name, date
#     LIMIT 10
# """
# # W Francji pierwsze zachorowania 22.01.2020
#
# query_job = client.query(query)
# daily_records_df = query_job.result().to_dataframe()
# daily_records_df

In [39]:
# # 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.
# query = """
#     SELECT column_name, data_type
#     FROM bigquery-public-data.covid19_open_data.INFORMATION_SCHEMA.COLUMNS
#     WHERE table_name = 'covid19_open_data'
#     limit 10
#
# """
#
# #Do prawdzania czy są jakieś dziwne wartośći w kolumnach z danymi
# #query = """
# #    SELECT MIN(new_confirmed) AS min_confirmed, MAX(new_confirmed) AS max_confirmed
# #    FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
# #    WHERE new_confirmed IS NOT NULL
# #"""
# #Dla new_confirmed: -5045418	5047094
#
# #query = """
# #    SELECT MIN(cumulative_confirmed) AS min_Cconfirmed, MAX(cumulative_confirmed) AS max_Cconfirmed
# #    FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
# #    WHERE cumulative_confirmed IS NOT NULL
# #"""
# #Dla comulative_confirmed: 0	92440495
#
# #query = """
# #    SELECT MIN(new_deceased) AS min_deceased, MAX(new_deceased) AS max_deceased
# #    FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
# #    WHERE new_deceased IS NOT NULL
# #"""
# #Dla new_deceased: -63811	63830
#
# #query = """
# #    SELECT MIN(cumulative_deceased) AS min_Cdeceased, MAX(cumulative_deceased) AS max_Cdeceased
# #    FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
# #    WHERE cumulative_deceased IS NOT NULL
# #"""
# #DLa cumulative_deceased: 0	1005195
#
# #query = """
# #    SELECT MIN(cumulative_tested) AS min_Ctested, MAX(cumulative_tested) AS max_Ctested
# #    FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
# #    WHERE cumulative_tested IS NOT NULL
# #"""
# #Dla cumulative_tested: 0	9214000000
#
# query_job = client.query(query)
# numeric_values_df = query_job.result().to_dataframe()
# numeric_values_df

In [40]:
# # 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków
# # nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.
#
# #Dla danych
# query = """
#     SELECT MIN(date) AS start_date, MAX(date) AS end_date
#     FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
# """
# #Dało wynik 2020-01-01  2022-09-17
#
# #Dla nowych zachorowań
# query = """
#     SELECT MIN(date) AS start_date, MAX(date) AS end_date
#     FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
#     WHERE new_confirmed IS NOT NULL AND new_confirmed != 0
# """
# #Dało wynik 2020-01-01  2022-09-15
#
# #Dla nowych śmierci
# query = """
#     SELECT MIN(date) AS start_date, MAX(date) AS end_date
#     FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
#     WHERE new_deceased IS NOT NULL AND new_deceased != 0
# """
# #Dało wynik 2020-01-02  2022-09-15
#
# #Dla nowych zaszczepionych
# query = """
#     SELECT MIN(date) AS start_date, MAX(date) AS end_date
#     FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
#     WHERE new_persons_vaccinated IS NOT NULL AND new_persons_vaccinated != 0
# """
# #Dało wynik 2020-12-07  2022-09-14
#
# query_job = client.query(query)
# time_range_df = query_job.result().to_dataframe()
# time_range_df

In [41]:
# # 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń.
# # Dodaliśmy limit 10, aby uruchomienie trwało szybciej
# #dane o hospitalizacji
# query = """
# select country_name, date, cumulative_hospitalized_patients, cumulative_intensive_care_patients, hospital_beds_per_1000
# from `bigquery-public-data.covid19_open_data.covid19_open_data`
# where cumulative_hospitalized_patients is not null
# limit 10
# """
# #informacje demograficzne o zakażeniach
# query = """
# select country_name, date, new_confirmed_male, new_confirmed_female
# from `bigquery-public-data.covid19_open_data.covid19_open_data`
# where population_male is not null and population_female is not null
# limit 10
# """
# #dane o testowaniu
# query = """
# select country_name, date, cumulative_tested, new_tested
# from `bigquery-public-data.covid19_open_data.covid19_open_data`
# where cumulative_tested is not null
# limit 10
# """
# #dane o szczepieniach
# query = """
# select country_name, date, cumulative_persons_fully_vaccinated, new_persons_fully_vaccinated
# from `bigquery-public-data.covid19_open_data.covid19_open_data`
# where cumulative_persons_fully_vaccinated is not null
# limit 10
# # """
# #zdrowie a zgony
# query = """
# select country_name, date, cumulative_deceased, health_expenditure_usd
# from `bigquery-public-data.covid19_open_data.covid19_open_data`
# where cumulative_deceased is not null and health_expenditure_usd is not null
# limit 10
# """
#
# query_job = client.query(query)
# data = query_job.result().to_dataframe()
# data


In [ ]:
# 4.0
query = """
    select location_key, date, country_code, country_name,
    new_confirmed, new_deceased, cumulative_confirmed, cumulative_deceased, new_tested, cumulative_tested,
    new_persons_vaccinated, cumulative_persons_vaccinated, new_recovered,
    new_persons_fully_vaccinated, cumulative_persons_fully_vaccinated,
    population, population_male, population_female,
    population_density, human_development_index, gdp_usd,
    gdp_per_capita_usd, area_sq_km, vaccination_policy, stringency_index,
    life_expectancy, new_hospitalized_patients
    from `bigquery-public-data.covid19_open_data.covid19_open_data` 
    where aggregation_level = 0
"""

query_job = client.query(query)
df = query_job.result().to_dataframe()

df

In [ ]:
#4.01 czyszczenie danych
import numpy as np

# 1. Ogólne czyszczenie danych

# Konwersja kolumny date do formatu daty
df['date'] = pd.to_datetime(df['date'])

# Usunięcie duplikatów
df_clean = df.drop_duplicates()

# Usuwamy wiersze bez kluczowych informacji (np. brak kodu lub nazwy kraju)
df_clean = df_clean.dropna(subset=['country_code', 'country_name'])

# Ujednolicenie: ustawienie kodu kraju na wielkie litery
#df_clean['country_code'] = df_clean['country_code'].str.upper()

# Sprawdzamy dane liczbowe – ujemne wartości traktujemy jako błędne i ustawiamy jako NaN
num_cols = ['population', 'population_density', 'human_development_index', 
           'gdp_usd', 'gdp_per_capita_usd', 'area_sq_km', 'life_expectancy']
for col in num_cols:
   df_clean.loc[df_clean[col] < 0, col] = np.nan

df_clean

In [63]:
#  4.1. Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i 
#  potencjalnie przyszłościowe do dalszego przetwarzania.

countries_cols = [
    'country_code', 'country_name', 'population', 'population_male', 'population_female', 'population_density',
    'human_development_index', 'gdp_usd', 'gdp_per_capita_usd', 'area_sq_km', 'life_expectancy'
]
df_countries = df_clean[countries_cols].drop_duplicates()
# Opcjonalnie: usuwamy wiersze, gdzie brakuje kluczowych informacji o kraju
df_countries = df_countries.dropna(subset=['population', 'population_density', 'gdp_usd'])
# Propozycja naprawy błędnych danych: np. dla kolumn numerycznych warto sprawdzić, czy nie występują wartości ujemne
# i w razie ich wystąpienia zastąpić je np. zerem lub wartością mediany (tutaj tylko komentarz)

# Zapis do pliku CSV
df_countries.to_csv('4_1_countries_data.csv', index=False)

#df_countries

,country_code,country_name,population,population_density,human_development_index,gdp_usd,gdp_per_capita_usd,area_sq_km,life_expectancy
0,AR,Argentina,44938712,16.515,0.825,449663446954,10006,2780400,76.52
9,BR,Brazil,212559409,25.431,0.754,1839758040765,8717,8515770,75.672
13,CO,Colombia,50882884,45.861,0.747,323802808108,6432,1141748,77.109
15,CZ,Czech Republic,10649800,137.700,0.888,244932977600,23010,78870,79.1
20,DE,Germany,83019213,234.700,0.942,3946356600000,47554,357580,81.0
24,EE,Estonia,1324820,30.400,0.892,30722303000,23246,45340,78.5
30,ES,Spain,46937060,93.100,0.876,1418587740000,30326,505935,83.5
35,FI,Finland,5517919,18.100,0.883,276654540000,50150,338450,81.8
42,FR,France,67012883,105.600,0.901,2776646200000,41300,549086,82.9
51,HK,Hong Kong,7496988,7139.989,0.933,366029556273,48755,1110,84.934146


In [64]:
# 4.2. Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

cases_cols = [
    'location_key', 'date', 'country_code', 'country_name', 
    'new_confirmed', 'cumulative_confirmed', 'new_tested', 'cumulative_tested',
    'new_recovered', 'new_hospitalized_patients'
]
df_cases = df_clean[cases_cols].drop_duplicates()
df_cases = df_cases.dropna(subset=['date', 'new_confirmed', 'cumulative_confirmed']) # Usuwanie wierszy z brakującymi wartosciami
df_cases.to_csv('4_2_global_covid_cases.csv', index=False)

#df_cases

,location_key,date,country_code,country_name,new_confirmed,cumulative_confirmed,cumulative_tested
0,AR,2021-07-24,AR,Argentina,9723,4902068,14689062
1,AR,2021-02-08,AR,Argentina,7383,2019751,5829132
2,AR,2021-01-16,AR,Argentina,8426,1832067,4940563
3,AR,2020-05-11,AR,Argentina,286,6648,85533
4,AR,2022-04-23,AR,Argentina,577,9076677,27682918
5,AR,2020-02-29,AR,Argentina,0,82,489
6,AR,2021-12-18,AR,Argentina,4453,5412387,20364853
7,AR,2020-04-22,AR,Argentina,122,3481,39188
8,AR,2021-08-14,AR,Argentina,5344,5131615,15919490
9,BR,2021-09-08,BR,Brazil,14430,20928008,53449516


In [65]:
# 4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

mortality_cols = [
    'location_key', 'date', 'country_code', 'country_name', 
    'new_deceased', 'cumulative_deceased'
]
df_mortality = df_clean[mortality_cols].drop_duplicates()
df_mortality = df_mortality.dropna(subset=['date', 'new_deceased', 'cumulative_deceased'])
df_mortality.to_csv('4_3_covid_mortality.csv', index=False)

#df_mortality

,location_key,date,country_code,country_name,new_deceased,cumulative_deceased
0,AR,2021-07-24,AR,Argentina,263,108373
1,AR,2021-02-08,AR,Argentina,120,52881
2,AR,2021-01-16,AR,Argentina,174,48949
3,AR,2020-05-11,AR,Argentina,15,386
4,AR,2022-04-23,AR,Argentina,2,128748
5,AR,2020-02-29,AR,Argentina,0,1
6,AR,2021-12-18,AR,Argentina,16,117285
7,AR,2020-04-22,AR,Argentina,11,183
8,AR,2021-08-14,AR,Argentina,178,112527
9,BR,2021-09-08,BR,Brazil,250,584421


In [66]:
# 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

vaccination_cols = [
    'location_key', 'date', 'country_code', 'country_name', 
    'new_persons_vaccinated', 'cumulative_persons_vaccinated', 
    'new_persons_fully_vaccinated', 'cumulative_persons_fully_vaccinated'
]
df_vaccinations = df_clean[vaccination_cols].drop_duplicates()
df_vaccinations = df_vaccinations.dropna(subset=['date'])
df_vaccinations.to_csv('4_4_vaccination_trends.csv', index=False)

#df_vaccinations

,location_key,date,country_code,country_name,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated
0,AR,2021-07-24,AR,Argentina,78415,10453823,22098,2688146
1,AR,2021-02-08,AR,Argentina,6917,149910,2805,88091
2,AR,2021-01-16,AR,Argentina,2518,99853,1,21
3,AR,2020-05-11,AR,Argentina,<NA>,<NA>,<NA>,<NA>
4,AR,2022-04-23,AR,Argentina,490,17610449,2188,15933366
5,AR,2020-02-29,AR,Argentina,<NA>,<NA>,<NA>,<NA>
6,AR,2021-12-18,AR,Argentina,10496,16424854,25107,13864476
7,AR,2020-04-22,AR,Argentina,<NA>,<NA>,<NA>,<NA>
8,AR,2021-08-14,AR,Argentina,15965,11705463,93899,4660412
9,BR,2021-09-08,BR,Brazil,607843,137222960,1178227,65009622


In [67]:
# 4.5. Zdefiniuj własny dodatkowy przypadek.

# Przykładowo: Analiza związku między restrykcjami rządowymi (stringency_index) a liczbą nowych zachorowań
# Można wybrać coś ciekawszego tho
df_custom = df_clean[['date', 'country_name', 'new_confirmed', 'stringency_index']].copy()
# Usuwamy rekordy, w których brakuje krytycznych informacji
df_custom = df_custom.drop_duplicates().dropna(subset=['date', 'new_confirmed', 'stringency_index'])
df_custom.to_csv('4_5_custom_case.csv', index=False)

#df_custom

,date,country_name,new_confirmed,stringency_index
0,2021-07-24,Argentina,9723,75.00
1,2021-02-08,Argentina,7383,79.17
2,2021-01-16,Argentina,8426,79.17
3,2020-05-11,Argentina,286,94.44
4,2022-04-23,Argentina,577,39.81
5,2020-02-29,Argentina,0,11.11
6,2021-12-18,Argentina,4453,41.67
7,2020-04-22,Argentina,122,100.00
8,2021-08-14,Argentina,5344,75.93
9,2021-09-08,Brazil,14430,59.72


In [71]:
#5.0
# Dataframey powstałe w części 4
# df_countries         - dane statyczne o krajach (4.1)
# df_cases             - dane dotyczące zachorowań (4.2)
# df_mortality         - dane dotyczące śmiertelności (4.3)
# df_vaccinations      - dane dotyczące szczepień (4.4)
# df_custom           - dane customowe(4.5)

# 1. Łączenie danych czasowych
# Klucz łączący to: "location_key", "date", "country_code", "country_name"

# Łączymy dane dotyczące zachorowań i śmiertelności
df_time_series = pd.merge(
    df_cases, df_mortality,
    on=["location_key", "date", "country_code", "country_name"],
    how="outer"
)

# Dołączamy dane dotyczące szczepień
df_time_series = pd.merge(
    df_time_series, df_vaccinations,
    on=["location_key", "date", "country_code", "country_name"],
    how="outer"
)

# Upewnij się, że kolumna 'date' w obu DataFrame’ach jest w formacie datetime64[ns]
df_time_series['date'] = pd.to_datetime(df_time_series['date'])
df_custom['date'] = pd.to_datetime(df_custom['date'])

# Dołączamy dane dotyczące testów
df_time_series = pd.merge(
    df_time_series, df_custom,
    on=["date", "country_name"],
    how="outer"
)

# df_countries nie zawiera kolumny 'date', dlatego łączymy je na podstawie wspólnych kluczy: "country_code" i "country_name"
df_final = pd.merge(
    df_time_series, df_countries,
    on=["country_code", "country_name"],
    how="left"
)

# Usuwamy ewentualne duplikaty
df_final = df_final.drop_duplicates()

# Sortujemy dane – najczęściej po kodzie kraju i dacie, by mieć ujednoliconą strukturę
df_final = df_final.sort_values(by=["country_code", "date"])

# Zapisujemy wynik do pliku CSV
df_final.to_csv("5.0_combined_covid_data.csv", index=False)

#df_final

,location_key,date,country_code,country_name,new_confirmed_x,cumulative_confirmed,cumulative_tested,new_deceased,cumulative_deceased,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_confirmed_y,stringency_index,population,population_density,human_development_index,gdp_usd,gdp_per_capita_usd,area_sq_km,life_expectancy
3,AR,2020-02-29,AR,Argentina,0,82,489,0,1,<NA>,<NA>,<NA>,<NA>,0,11.11,44938712,16.515,0.825,449663446954,10006,2780400,76.52
8,AR,2020-04-22,AR,Argentina,122,3481,39188,11,183,<NA>,<NA>,<NA>,<NA>,122,100.00,44938712,16.515,0.825,449663446954,10006,2780400,76.52
9,AR,2020-05-11,AR,Argentina,286,6648,85533,15,386,<NA>,<NA>,<NA>,<NA>,286,94.44,44938712,16.515,0.825,449663446954,10006,2780400,76.52
46,AR,2021-01-16,AR,Argentina,8426,1832067,4940563,174,48949,2518,99853,1,21,8426,79.17,44938712,16.515,0.825,449663446954,10006,2780400,76.52
47,AR,2021-02-08,AR,Argentina,7383,2019751,5829132,120,52881,6917,149910,2805,88091,7383,79.17,44938712,16.515,0.825,449663446954,10006,2780400,76.52
70,AR,2021-07-24,AR,Argentina,9723,4902068,14689062,263,108373,78415,10453823,22098,2688146,9723,75.00,44938712,16.515,0.825,449663446954,10006,2780400,76.52
74,AR,2021-08-14,AR,Argentina,5344,5131615,15919490,178,112527,15965,11705463,93899,4660412,5344,75.93,44938712,16.515,0.825,449663446954,10006,2780400,76.52
84,AR,2021-12-18,AR,Argentina,4453,5412387,20364853,16,117285,10496,16424854,25107,13864476,4453,41.67,44938712,16.515,0.825,449663446954,10006,2780400,76.52
96,AR,2022-04-23,AR,Argentina,577,9076677,27682918,2,128748,490,17610449,2188,15933366,577,39.81,44938712,16.515,0.825,449663446954,10006,2780400,76.52
18,BR,2020-07-24,BR,Brazil,55891,2343366,8229249,1156,85238,<NA>,<NA>,0,0,55891,81.02,212559409,25.431,0.754,1839758040765,8717,8515770,75.672


In [72]:
#6.0
# 1. Wczytanie danych z części 5
df_covid = pd.read_csv("5.0_combined_covid_data.csv", parse_dates=["date"])

# 2. Wczytanie danych z world_countries.csv
df_world = pd.read_csv("world_countries.csv")
# Zmieniamy nazwę kolumny "Country/Territories" na "country_name" dla ułatwienia łączenia
df_world.rename(columns={"Country/Territory": "country_name"}, inplace=True)

# 3. Wczytanie danych z GDP.csv
df_gdp = pd.read_csv("GDP.csv")
# Zmieniamy nazwę kolumny "Country Name" na "country_name"
df_gdp.rename(columns={"Country Name": "country_name"}, inplace=True)

# 4. Normalizacja kluczy łączenia (pełne nazwy krajów) we wszystkich DataFrame'ach
df_covid["country_name"] = df_covid["country_name"].str.strip().str.lower()
df_world["country_name"] = df_world["country_name"].str.strip().str.lower()
df_gdp["country_name"] = df_gdp["country_name"].str.strip().str.lower()

# 5. Dla danych covidowych wyodrębniamy rok z kolumny "date", by móc łączyć z danymi PKB
df_covid["year"] = df_covid["date"].dt.year

# 6. Łączenie danych
# 6.1. Łączymy dane covidowe z danymi krajowymi (world_countries.csv) – łączymy po "country_name"
df_merged = pd.merge(df_covid, df_world, on="country_name", how="left")

# 6.2. Łączymy wynik z danymi PKB. W df_gdp kolumna z rokiem to zazwyczaj "Year", więc łączymy:
df_merged = pd.merge(df_merged, df_gdp, left_on=["country_name", "year"], 
                     right_on=["country_name", "Year"], how="left")
# Połączone dane mają już dwie kolumny określające rok, więc możemy usunąć zbędną kolumnę "Year":
df_merged.drop(columns=["Year"], inplace=True)

# 7. Usunięcie zbędnych kolumn: 'area_sq_km', 'Country Code', 'Value'
df_merged = df_merged.drop(columns=['area_sq_km', 'Country Code', 'Value'], errors='ignore')

# 8. Finalne czyszczenie danych – usunięcie duplikatów itp.
df_merged.drop_duplicates(inplace=True)

df_pivoted = df_merged.pivot(['country_code', 'country_name'],)
# 9. Zapisanie wyniku do pliku CSV
df_pivoted.to_csv("6.0_combined_covid_world_gdp_data.csv", index=False)

#Dane z części 5 oraz pliki world_countries.csv i GDP.csv zostały połączone i zapisane do pliku combined_covid_world_gdp_data.csv."


Dane z części 5 oraz pliki world_countries.csv i GDP.csv zostały połączone i zapisane do pliku combined_covid_world_gdp_data.csv.
